In [ ]:
# Importing dataset and libraries

import pandas as pd

df = pd.read_csv("data_titanic.csv")
df.head(3)

In [ ]:
print("How many people died? -->", len(df[df["Survived"]==0]))

In [ ]:
# Setting up the LLM

!pip install langchain_community
from langchain_community.llms import Ollama #0.2.12

llm = Ollama(model="llama3.1")

result = llm.invoke(input=['''Do you know the Titanic dataset? If yes, tell me how many people died.''']).split("\n")[0]
print(result)

In [ ]:
# Connecting DB

import sqlite3
df.to_sql(index=False, name="dataset", con=sqlite3.connect("database.db"), if_exists="replace")

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///database.db")
db

In [ ]:
# Creating an agent to check out the SQL Commands

!pip install crewai_tools

from crewai_tools import tool #0.8.3
from langchain_community.tools.sql_database.tool import ListSQLDatabaseTool, InfoSQLDatabaseTool, QuerySQLDataBaseTool

@tool("table")
def table() -> str:
    """Get all the tables in the database"""
    return ListSQLDatabaseTool(db = db).invoke("")

@tool("tool_schema")
def tool_schema(tables: str) -> str:
    """Get the table schema. Example Input: t1, t2, t3"""
    tool = InfoSQLDatabaseTool(db = db)
    return tool.invoke(tables)

@tool("tool_query")
def tool_query(sql: str) -> str:
    """Run a SQL query"""
    return QuerySQLDataBaseTool(db = db).invoke(sql)

print("Get Tables")
print(table.run())

print("\nGet Schema")
print(tool_schema.run(tables = table.run()))

print("\nSQL Query")
print(tool_query.run(f"SELECT*FROM {table.run()} LIMIT 3"))

In [ ]:
!pip install langchain_community

# Checking SQL with LLM

from langchain_community.tools.sql_database.tool import QuerySQLCheckerTool

@tool("tool_check")
def tool_check(sql: str) -> str:
    """
    Before executing a query, use this tool to review the SQL query.
    """
    return QuerySQLCheckerTool(db = db, llm = llm).invoke({"query":sql})

tool_check.run(f"SELECT*FROM {tool_tables.run()} LIMIT 3 +db+")

In [ ]:
!pip install crewai
import crewai #0.51.1

prompt = '''Extract data with an SQL query to answer {user_input}'''

# Defining Agent
agent_sql = crewai.Agent(
    role="Database Engineer",
    goal=prompt,
    backstory='''
        You are an experienced database engineer that creates and optimize efficient SQL queries.
        Use `tables` to find tables.
        Use `tool_schema` to get the metadata for the tables.
        Use `tool_check` to review your queries before executing.
        Use `tool_query` to execute SQL queries.
     ''',
    tools=[tables, tool_schema, tool_query, tool_check],
    max_iterations = 10,
    llm = llm,
    allow_delegation = False, verbose = True)

# Defining Task
task_sql = crewai.Task(
    description = prompt,
    agent = agent_sql,
    expected_output = '''Output of the query''')

In [ ]:
# Creating an agent for Python

!pip install langchain_experimental
!pip install langchain_core
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool

tool_pycode = Tool(name="tool_pycode",
    description='''
    A Python shell. Use this to execute python commands. Input should be a valid python command.
    If you want to see the output of a value, you should print it out with `print(...)`.
    ''',
    func=PythonREPL().run)

print(tool_pycode.run("import numpy as np; print(np.sum([1,2]))") )

In [ ]:
## LLM Checking Python

@tool("tool_eval")
def tool_eval(code: str) -> str:
    """
    Before executing Python code, always use this tool to evaluate code and correct the code if necessary.
    Example: `import numpy as np print(np.sum([1,2]))` would give an error,
    so you must change it to `import numpy as np; print(np.sum([1,2]))`
    """
    res = llm.invoke(input=[f'''review the following python code and correct it if you find errors.
    You must return only the correct code: {code}''']).split("\n")[0]
    return res

print(tool_eval.run("import numpy as np") )

In [ ]:
prompt = '''You analyze the data received from the database engineer to answer {user_input}'''

## Agent
agent_py = crewai.Agent(
    role="Data Analyst",
    goal=prompt,
    backstory='''
        You are an experienced data anlyst that analyzes datasets using Python.
        You have attention to detail and always produce very clear and detailed results.
        First generate Python code you need to analyze the data.
        Then use the `tool_eval` to check your code.
        Finally use `tool_pycode` to execute the code and return the output.
    ''',
    tools = [tool_eval, tool_pycode],
    max_iterations = 10,
    llm = llm,
    allow_delegation = False, verbose = True)

## Task
task_py = crewai.Task(
    description = prompt,
    agent = agent_py,
    context = [task_sql],
    expected_output = '''Output of Python code''')

In [ ]:
# Creating HTML Agent

prompt = '''You write executive summary reports based on the work of the data analyst to answer {user_input}'''

## Agent
agent_html = crewai.Agent(
    role = "Web Developer",
    goal = prompt,
    backstory = '''
        You are an experienced web developer that writes beautiful reports using HTML and CSS.
        You always summarize texts into bullet points containing the most important details.
        At the end add an interactive button with JavaScript so the user can approve the report,
        and if the user clicks the button, show a pop-up text.
     ''',
    max_iterations = 10,
    llm = llm,
    allow_delegation = False, verbose = True)

## Task
task_html = crewai.Task(
    description = prompt,
    agent = agent_html,
    context = [task_py],
    expected_output = '''HTML code''')

In [ ]:
# Using multiple agents at once to check the difference in outputs

crew = crewai.Crew(agents = [agent_sql, agent_py, agent_html],
                   tasks = [task_sql, task_py, task_html],
                   process = crewai.Process.sequential,
                   verbose = True)

res = crew.kickoff(inputs = {"user_input":"how many people died?"})

print("\nResult\n", res)